In [14]:
import itertools

from nltk.lm import Vocabulary
from nltk.lm.models import MLE
from nltk.util import ngrams

In [15]:
N = 3
filepath = "text/peachy/peachy-4304732.txt"

In [8]:
sentences = []
with open(filepath, "r") as f:
    for line in f.readlines()[2:]:
        s = line.strip()
        if len(s) == 0:
            continue
        sentences.append(s)

In [23]:
print("\n".join(sentences))

女性を潤す新たな注目ワードは“アミノ酸”
8月も残すところ10日余り。秋のはじまりと共に気になってくるのがお肌の乾燥ケアだ。潤いとハリのある肌の為にヒアルロン酸とコラーゲンに着目した「ベキュア スキンパーフェクトスフィア」シリーズに、「アミノ酸」に着目したニューアイテムが登場した。
皮膚成分のほとんどはアミノ酸によって占められており、素肌は良質なアミノ酸を常に求めている。今回新シリーズは、アルギニン、保湿作用のあるグルタミン酸、角質を形成するセリン、ハリ感をサポートするプロリンをカプセルに内包し、肌に速攻アプローチ。
特に、ベキュア初となるバームタイプのメイク落としは、毛穴の奥の汚れまですっきり落としつつ、マッサージもできるという1品2役の楽ちんコスメ。マッサージだけだと続けるのは難しいが、毎日行うクレンジングと共に肌に溜まった汚れの排出を促し、すっきりとした輪郭とワントーン明るい肌を手に入れることができる。
また、十分な睡眠時間がとれず、肌本来の再生能力が不足している現代女性の為に睡眠中の肌潤い生理に注目したナイトクリームも登場。眠っている間に潤いとハリ感を与え、朝目覚めた時には艶めく素肌が手に入る。
今年の秋の乾燥ケアにはアミノ酸。忙しい日々で失いがちな潤いを、手軽に補充し、艶肌を手に入れよう。アミノアクトクレンジングバーム、アミノアクトクレンジングフォーム、アミノエフェクティブスリープは8月20日より販売開始。
・ベキュア オンラインショップ


In [16]:
vocabulary = Vocabulary(itertools.chain.from_iterable(sentences))
char_ngram = [ngrams(sentence, N) for sentence in sentences]

In [19]:
list(ngrams(sentences[0], N))[:3]

[('女', '性', 'を'), ('性', 'を', '潤'), ('を', '潤', 'す')]

In [21]:
lm = MLE(order=N, vocabulary=vocabulary)
lm.fit(char_ngram)

In [30]:
context = ("睡", "睡")
for word in lm.context_counts(lm.vocab.lookup(context)):
    print(f"{word}: {lm.score(word, context)}")

In [39]:
text = "アミノ酸と睡眠時間がある。"
list(ngrams(text, N))[:3]

[('ア', 'ミ', 'ノ'), ('ミ', 'ノ', '酸'), ('ノ', '酸', 'と')]

In [40]:
test_contexts = list(ngrams(text, N))

In [47]:
scores = [0 for _ in range(len(text))]
i = 0
tp = 0
for context in test_contexts:
    p = lm.score(context[-1], context[:-1])
    print(f"{context}: {p}")
    if p <= tp:
        for j in range(N):
            scores[i+j] -= 1
    i+=1

('ア', 'ミ', 'ノ'): 1.0
('ミ', 'ノ', '酸'): 0.625
('ノ', '酸', 'と'): 0.0
('酸', 'と', '睡'): 0.0
('と', '睡', '眠'): 0
('睡', '眠', '時'): 0.5
('眠', '時', '間'): 1.0
('時', '間', 'が'): 1.0
('間', 'が', 'あ'): 0.0
('が', 'あ', 'る'): 0
('あ', 'る', '。'): 0.0


In [49]:
ts = -2
for c, score in zip(list(text), scores):
    if score <= ts:
        print(f"{c}: {score}  # error!")
    else:
        print(f"{c}: {score}  # ok")

ア: 0  # ok
ミ: 0  # ok
ノ: -1  # ok
酸: -2  # error!
と: -3  # error!
睡: -2  # error!
眠: -1  # ok
時: 0  # ok
間: -1  # ok
が: -2  # error!
あ: -3  # error!
る: -2  # error!
。: -1  # ok
